# NN Assignment 1

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import average_precision_score

# Task 13
# Use object oriented programming and/or the Matrix Notation to build and train the network
# 2 classes were created as part of the Object Oriented Programming, the NeuronLayer class representing neurons and the NueralNetwork class representing the whole Nueral Network


class NeuronLayer:
    def __init__(self, input_size, output_size):
        self.input_size = input_size
        self.output_size = output_size

        # Initialize weights and biases
        self.weights = np.random.randn(input_size, output_size)
        self.biases = np.zeros((1, output_size))
        self.input = None
        self.output = None

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        return x * (1 - x)

    def forward_pass(self, X):
        # Forward pass
        self.input = X
        self.output = self.sigmoid(np.dot(X, self.weights) + self.biases)

    def backward_pass(self, error, learning_rate):
        # Backward pass (compute gradients)
        delta = error * self.sigmoid_derivative(self.output)

        # Update weights using gradient descent
        self.weights += self.input.T.dot(delta) * learning_rate

class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        # Create neuron layers
        self.hidden_layer = NeuronLayer(input_size, hidden_size)
        self.output_layer = NeuronLayer(hidden_size, output_size)

    # Task 8
    # Error Aggrigation
    # Calculate the aggrigated error
    def error_Nuweiba(self, actual_outputs, target_outputs):
        # Calculate error vector
        error_vector = actual_outputs - target_outputs

        # Calculate aggregated error
        aggregated_error = np.mean(np.abs(error_vector))

        return error_vector, aggregated_error

    def forward_pass(self, X):
        # Forward pass
        self.hidden_layer.forward_pass(X)
        self.output_layer.forward_pass(self.hidden_layer.output)

    def compute_loss(self, y):
        # Compute the loss
        return np.mean(0.5 * (y - self.output_layer.output) ** 2)

    # Task 6
    # Implement the backpropagation algorithm to train the network. The function apply one training epoch only.
    def backward_pass(self, X, y, learning_rate):
        # Backward pass (compute gradients)
        output_error = y - self.output_layer.output

        # Backpropagation through layers
        self.output_layer.backward_pass(output_error, learning_rate)
        self.hidden_layer.backward_pass(output_error.dot(self.output_layer.weights.T), learning_rate)

    def validate(self, X_val, y_val):
        # Make predictions on the validation set
        self.forward_pass(X_val)
        predictions = (self.output_layer.output > 0.5).astype(int)

        # Calculate validation accuracy
        accuracy = np.mean(predictions == y_val)
        return accuracy

    def calculate_map(self, predictions, ground_truth):
      # Calculate mean average precision
        mAP = average_precision_score(ground_truth, predictions)
        return mAP

    # Task 7
    # Impalement a function to test the network by forward passing a set of labeled data
    def test_Nuweiba(self, X_test, y_test):
        # Make predictions on the test set
        self.forward_pass(X_test)
        predictions = (self.output_layer.output > 0.5).astype(int)

        # Calculate test accuracy
        accuracy = np.mean(predictions == y_test)
        print(f"Test Accuracy: {accuracy}")

        # Confusion Matrix
        conf_matrix = confusion_matrix(y_test, predictions)
        print("Confusion Matrix:")
        print(conf_matrix)

        # Plot Confusion Matrix
        self.plot_confusion_matrix(conf_matrix)

        # Calculate error matrix and aggregated error
        error_vector, aggregated_error = self.error_Nuweiba(predictions, y_test)

        # Print and return error information
        print(f"Aggregated Error: {aggregated_error}")

        return error_vector, aggregated_error

    def train_Nuweiba(self, X, y, X_val, y_val, epochs, learning_rate, batch_size=100):
        num_samples = X.shape[0]
        training_loss_history = []
        validation_accuracy_history = []
        map_history = []
        aggregated_error_history = []

        # Initialize the final weights matrix
        final_weights_matrix = None

        for epoch in range(epochs):
            # Shuffle the data before each epoch
            indices = np.arange(num_samples)
            np.random.shuffle(indices)
            X_shuffled = X[indices]
            y_shuffled = y[indices]

            for i in range(0, num_samples, batch_size):
                # Get the current batch
                X_batch = X_shuffled[i:i+batch_size]
                y_batch = y_shuffled[i:i+batch_size]

                # Forward pass
                self.forward_pass(X_batch)

                # Compute the loss
                loss = self.compute_loss(y_batch)

                # Task 9
                # The backpropagation pass should be executed after each iteration (one batch).
                # Backward pass
                self.backward_pass(X_batch, y_batch, learning_rate)

            # Task 10
            # The validation is performed after each epoch.
            # Validate after every epoch
            validation_accuracy = self.validate(X_val, y_val)
            validation_accuracy_history.append(validation_accuracy)

            # Calculate and save mAP for demonstration purposes
            predictions = self.output_layer.output
            ground_truth = y_val
            map_value = self.calculate_map(predictions, ground_truth)
            map_history.append(map_value)

            # Calculate and save aggregated error
            error_vector, aggregated_error = self.error_Nuweiba(predictions, ground_truth)
            aggregated_error_history.append(aggregated_error)

            # Print the loss, validation accuracy, and mAP after every epoch
            print(f"Epoch {epoch + 1}, Loss: {loss}, Validation Accuracy: {validation_accuracy}, mAP: {map_value}, Aggregated Error: {aggregated_error}")

            # Save training loss for plotting
            training_loss_history.append(loss)

            # Save the weights matrix for the final epoch
            final_weights_matrix = self.output_layer.weights.copy()

        # Plotting
        self.plot_stats(training_loss_history, validation_accuracy_history, map_history, aggregated_error_history)

        # Return the final weights matrix
        return final_weights_matrix

    # Task 12
    # Report the training, validation and testing accuracies. Use tables, graphs, and charts as possible
    # Plotting functions
    def plot_confusion_matrix(self, conf_matrix):
        plt.figure(figsize=(6, 6))
        plt.imshow(conf_matrix, interpolation='nearest', cmap=plt.cm.Blues)
        plt.title('Confusion Matrix')
        plt.colorbar()

        classes = ['Class 0', 'Class 1']  # Adjust classes based on your actual classes
        tick_marks = np.arange(len(classes))
        plt.xticks(tick_marks, classes, rotation=45)
        plt.yticks(tick_marks, classes)

        plt.ylabel('True label')
        plt.xlabel('Predicted label')
        plt.show()

    def plot_stats(self, training_loss_history, validation_accuracy_history, map_history, aggregated_error_history):
        # Plot training loss, validation accuracy, mAP, and aggregated error
        plt.figure(figsize=(18, 5))

        # Plot training loss
        plt.subplot(1, 4, 1)
        plt.plot(training_loss_history, label='Training Loss')
        plt.title('Training Loss Over Epochs')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()

        # Plot validation accuracy
        plt.subplot(1, 4, 2)
        plt.plot(validation_accuracy_history, label='Validation Accuracy', color='orange')
        plt.title('Validation Accuracy Over Epochs')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend()

        # Plot mAP
        plt.subplot(1, 4, 3)
        plt.plot(map_history, label='mAP', color='green')
        plt.title('mAP Over Epochs')
        plt.xlabel('Epoch')
        plt.ylabel('mAP')
        plt.legend()

        # Plot aggregated error
        plt.subplot(1, 4, 4)
        plt.plot(aggregated_error_history, label='Aggregated Error', color='red')
        plt.title('Aggregated Error Over Epochs')
        plt.xlabel('Epoch')
        plt.ylabel('Aggregated Error')
        plt.legend()

        plt.tight_layout()
        plt.show()

In [ ]:
# Task 2
# Use Banknote authentication dataset
# Importing dataset and creating Features and Labels for training and testing the Nueral Network

# Load the dataset and preprocess
# Load the dataset and preprocess
url1 = "https://archive.ics.uci.edu/ml/machine-learning-databases/00267/data_banknote_authentication.txt"
url2 = "/content/smoking.csv"
url3 = "/content/heart_disease_health_indicators_BRFSS2015.csv"

column_names1 = ["Variance", "Skewness", "Curtosis", "Entropy", "Class"]
column_names2 = ["ID","gender","age","height(cm)","weight(kg)","waist(cm)","eyesight(left)","eyesight(right)","hearing(left)","hearing(right)","systolic","relaxation","fasting blood sugar","Cholesterol","triglyceride","HDL","LDL","hemoglobin","Urine protein","serum creatinine","AST","ALT","Gtp","oral","dental caries","tartar","smoking"]
column_names3 = ["HeartDiseaseorAttack","HighBP","HighChol","CholCheck","BMI","Smoker","Stroke","Diabetes","PhysActivity","Fruits","Veggies","HvyAlcoholConsump","AnyHealthcare","NoDocbcCost","GenHlth","MentHlth","PhysHlth","DiffWalk","Sex","Age","Education","Income"]

df1 = pd.read_csv(url1, names=column_names1)
df2 = pd.read_csv(url2, names=column_names2)
df3 = pd.read_csv(url3, names=column_names3)

X = df1.drop("Class", axis=1).values
y = df1["Class"].values.reshape(-1, 1)

# Second Dataset
#X = df3.drop(["HeartDiseaseorAttack","HighBP","HighChol","CholCheck","BMI","Fruits","Veggies","HvyAlcoholConsump","AnyHealthcare","NoDocbcCost","GenHlth","MentHlth","PhysHlth","DiffWalk","Sex","Age","Education","Income"], axis=1).values
#y = df3["HeartDiseaseorAttack"].values.reshape(-1, 1)

#X = X[1:].astype(float).astype(int)
#y = y[1:].astype(float).astype(int)

#-----------------------------------------------------------------------------------------------------------------------------------------------#

# Task 3
# Split the data randomly into [T, V, S]. T = 70% for training, V = 20% for validation, and S = 10% for testing
# Spliting Data first 70% Training and 30% rest, then the 30% is divided 2/3 for Validation and 1/3 for Testing

# Split the data into training, validation, and testing sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.33, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

#-----------------------------------------------------------------------------------------------------------------------------------------------#

# Task 1
# Build a network as [4,10,1]. Input layer: 4 neurons, one hidden layer of 10 neurons and one output layer with 1 neuron
# Initialize the architecture values as mentioned

# Network Architecture [4, 10, 1]
input_size = 4
hidden_size = 10
output_size = 1

# Hyperparameters
learning_rate = 0.01
epochs = 10

# Create an instance of the NeuralNetwork class
neural_network = NeuralNetwork(input_size, hidden_size, output_size)

# Task 5
# Train the network at least 10 epochs
# Train the neural network with batch training and validate after each epoch
Weights_Matrix = neural_network.train_Nuweiba(X_train_scaled, y_train, X_val_scaled, y_val, epochs, learning_rate, batch_size=100)

print(Weights_Matrix)

#-----------------------------------------------------------------------------------------------------------------------------------------------#
# task 11
# The testing is performed only once after the completion of the training
# Testing the NeuralNetwork using the features and labels dedicated for testing (10% of the dataset)

# Test the neural network on the test set
Error_Vector, Error = neural_network.test_Nuweiba(X_test_scaled, y_test)

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import random
import itertools
import matplotlib.animation as animation
import math

# Define problem constraints and parameters
number_of_generations = 25  # Number of generations to run
generations_threshold = 5  # Number of generations to check if the fitness is getting worse
objective_function1_threshold = 1  # Global Maximum for objective function 1
objective_function2_threshold = -2  # Global Minimum for objective function 2
population = 10 # Population Size
mutation_rate = 0.005  # Mutation Rate
crossover_rate = 0.8 # The percentage of the population where crossover takes place

global_minimum = -1.75 # Global minimum for objective function 2	
global_maximum = 1.0  # Global maximum for objective function 1


generations = []  # List to store each generation
counter = 1  # Counter for tracking the generation

# Define the first objective function
def objective_function1(x):
    return math.sin(math.pi * x / 256)

# Define the second objective function
def objective_function2(x, y):
    return (x - 3.14)**2 + (y - 2.72)**2 + np.sin(3*x + 1.41) + np.sin(4*y - 1.73)

# Function to convert an integer to a binary string without the '0b' prefix
def int_to_binary(integer):
    if integer >= 0:
        sign_bit = '0'
    else:
        sign_bit = '1'
        integer = abs(integer)

    binary_repr = format(integer, '03b')  # Convert to 3-bit binary without '0b' prefix
    return sign_bit + binary_repr

# Function to convert a binary string to an integer
def binary_to_int(binary_str):
    if len(binary_str) != 4:
        return int(binary_str, 2)  # Convert to integer directly if the binary string is not Gray Code
    else:
        sign_bit = binary_str[0]
        binary_value = binary_str[1:]

        if sign_bit == '0':
            integer_value = int(binary_value, 2)
        else:
            integer_value = -int(binary_value, 2)

        return integer_value

# Convert a list of binary representations to a list of binary strings
def list_to_bin(l):
    y_b = []  # Initialize an empty list to store binary strings

    for i in range(len(l)):
        a = int_to_binary(l[i][0])  # Convert the first element of the pair to a binary string
        b = int_to_binary(l[i][1])  # Convert the second element of the pair to a binary string
        c = [a, b]  # Create a new list containing both binary strings
        y_b.append(c)  # Append the list of binary strings to the result list

    return y_b  # Return the list of binary strings

# Convert a list of binary or integer representations to a list of integer values
def list_to_int(l):
    y_b = []  # Initialize an empty list to store integer values

    for i in range(len(l)):
        if len(l[i]) == 2:  # Check if each element is a pair (two binary strings)
            a = binary_to_int(l[i][0])  # Convert the first binary string to an integer
            b = binary_to_int(l[i][1])  # Convert the second binary string to an integer
            c = [a, b]  # Create a new list containing both integer values
            y_b.append(c)  # Append the list of integer values to the result list
        else:
            a = binary_to_int(l[i])  # Convert a single binary string to an integer
            y_b.append(a)  # Append the integer value to the result list

    return y_b  # Return the list of integer values

# Convert a binary string to its Gray Code representation
def binary_to_gray(binary):
    if len(binary) == 4:  # Check if the binary string is of length 1
        sign_bit = binary[0]  # Store the sign bit
        gray = binary[1]  # Store the first bit of the Gray Code representation

        for i in range(2, len(binary)):
            gray_bit = int(binary[i]) ^ int(binary[i - 1])
            gray += str(gray_bit)

        if valid_gray(gray):
            return sign_bit + gray
        else:
            binary_to_gray(binary)

    else:
        gray = binary[0]
        for i in range(1, len(binary)):
            gray_bit = int(binary[i]) ^ int(binary[i - 1])
            gray += str(gray_bit)
        return gray

# Convert a list of binary representations to a list of Gray Code representations
def list_to_gray(g):
    y_g = []  # Initialize an empty list to store Gray Code strings

    for i in range(len(g)):
        if len(g[i]) == 2:  # Check if each element is a pair (two binary strings)
            a = binary_to_gray(g[i][0])  # Convert the first binary string to Gray Code
            b = binary_to_gray(g[i][1])  # Convert the second binary string to Gray Code
            c = [a, b]  # Create a new list containing both Gray Code strings
            y_g.append(c)  # Append the list of Gray Code strings to the result list
        else:
            a = binary_to_gray(g[i])
            y_g.append(a)

    return y_g  # Return the list of Gray Code strings

# Sort a list of binary strings in descending order based on the fitness of objective function 1
def sort_binary_descending(binary_lists):
    for i in range(len(binary_lists)):
        max_index = i
        # Compare fitness values of binary representations
        for j in range(i + 1, len(binary_lists)):
            if objective_function1(binary_to_int(binary_lists[j])) > objective_function1(binary_to_int(binary_lists[max_index])):
                max_index = j
        # Swap the binary representations to sort in descending order
        binary_lists[i], binary_lists[max_index] = binary_lists[max_index], binary_lists[i]
    return binary_lists  # Return the sorted list

# Sort a list of binary or integer representations in ascending order based on the fitness of objective function 2
def sort_ascending(list):
    for i in range(len(list)):
        min_index = i
        # Compare fitness values of integer representations
        for j in range(i + 1, len(list)):
            if objective_function2(list[j][0], list[j][1]) < objective_function2(list[min_index][0], list[min_index][1]):
                min_index = j
        # Swap the integer representations and convert back to binary
        list[i], list[min_index] = list[min_index], list[i]
    return list  # Return the sorted list

# CROSS OVER FUNCTION USING TYPE APPLICABLE TO BOTH REPRESENTATIONS
def crossover(parent1, parent2, type):
    if type == "uniform":  # Uniform crossover

        if random.random() < 0.5:
            child1 = parent1
            child2 = parent2
        else:
            child1 = parent2
            child2 = parent1

        return child1, child2

    elif type == "one_point":  # One-point crossover
        crossover_point = random.randint(1, len(parent1) - 1)
        child1 = parent1[:crossover_point] + parent2[crossover_point:]
        child2 = parent2[:crossover_point] + parent1[crossover_point:]
        child1 = ''.join(child1)
        child2 = ''.join(child2)
        return child1, child2  # Return the two children after one-point crossover

    elif type == "two_point":  # Two-point crossover
        crossover_points = sorted(random.sample(range(1, len(parent1) - 1), 2))
        child1 = parent1[:crossover_points[0]] + parent2[crossover_points[0]:crossover_points[1]] + parent1[crossover_points[1]:]
        child2 = parent2[:crossover_points[0]] + parent1[crossover_points[0]:crossover_points[1]] + parent2[crossover_points[1]:]
        child1 = ''.join(child1)
        child2 = ''.join(child2)
        return child1, child2  # Return the two children after two-point crossover


def mutation_objective2(individual):
    mutated_individual = 0
    mutated_gene = 0
    mutation_prob=0.5
    mutation_range=0.1

    if random.uniform(0,1) < mutation_prob:
        # Apply mutation: Add a random value within mutation_range to the gene
        mutated_gene = individual + random.uniform(-mutation_range, mutation_range)
        # Ensure the mutated gene stays within the range of -5 to 5
        mutated_gene = max(min(mutated_gene, 5), -5)
        mutated_individual = mutated_gene
    else:
        mutated_individual = individual

    return mutated_individual

# Mutation
def mutation(parent):
    mutated_parent = list(parent)  # Convert the binary string to a list for easy mutation
    for i in range(len(parent)):
        mutation_variable = random.random()
        if(mutation_variable <= mutation_rate):  # Check if a mutation should occur based on the mutation rate
            mutated_parent[i] = '1' if parent[i] == '0' else '0'  # Flip the bit with probability mutation_rate

    mutated_parent = ''.join(mutated_parent)  # Convert the list back to a binary string

    if len(mutated_parent) == 4:  # Check if the binary string is of length 4 (Gray Code)
        is_valid = valid_mutation(mutated_parent)  # Check if the mutated value is valid

        if is_valid:
            return ''.join(mutated_parent)  # Return the valid mutated value
        else:
            mutation(parent)  # Retry mutation if the value is not valid

    return mutated_parent  # Return the mutated binary string

# Check if the mutated binary value is valid for the problem constraints
def valid_mutation(parent):
    helper = parent[1:]  # Remove the sign bit
    val = binary_to_int(helper)  # Convert the remaining binary value to an integer
    if val >= -5 and val <= 5:  # Check if the integer value is within the specified range
        return True  # The mutation is valid
    else:
        return False  # The mutation is not valid
    

def valid_gray(parent):
    helper = parent  # Remove the sign bit
    val = binary_to_int(helper)  # Convert the remaining binary value to an integer
    if val >= -5 and val <= 5:  # Check if the integer value is within the specified range
        return True  # The mutation is valid
    else:
        return False  # The mutation is not valid  
    
    # Initialize Population
def initialize_population(pop_size, representation):
    population = []
    if representation == 'binary':
        for _ in range(pop_size):
            chromosome = [random.randint(0, 1) for _ in range(8)]  # 8-bit binary representation
            chromosome = ''.join(str(gene) for gene in chromosome)
            population.append(chromosome)
    elif representation == 'real':
        for _ in range(pop_size):
            x = random.uniform(-5, 5)
            y = random.uniform(-5, 5)
            population.append([x, y])
    return population


# Function to select a mating pool based on fitness scores
def select_mating_pool(population, fitness_fn, num_parents):
    fitness_scores = [fitness_fn(chromosome) for chromosome in population]  # Calculate fitness scores for each individual in the population
    selected_indices = np.argsort(fitness_scores)[-num_parents:]  # Select the indices of the top 'num_parents' individuals with the highest fitness scores
    mating_pool = [population[i] for i in selected_indices]  # Create the mating pool by selecting the individuals with the highest fitness scores
    return mating_pool  # Return the selected mating pool

# Function to visualize fitness values for Objective Function 1
def visualize_generations1(fitness_values):
    plt.figure()  # Create a new figure for plotting
    plt.plot(fitness_values)  # Plot the fitness values over generations
    plt.title("GA Generations Visualization - Objective Function 1")  # Set the title for the plot
    plt.xlabel("Generation Individuals")  # Set the x-axis label
    plt.ylabel("Fitness Value")  # Set the y-axis label
    plt.show()  # Display the plot

# Function to visualize fitness values for Objective Function 2
def visualize_generations2(fitness_values):
    plt.figure()  # Create a new figure for plotting
    plt.plot(fitness_values)  # Plot the fitness values over generations
    plt.title("GA Generations Visualization - Objective Function 2")  # Set the title for the plot
    plt.xlabel("Generation Individuals")  # Set the x-axis label
    plt.ylabel("Fitness Value")  # Set the y-axis label
    plt.show()  # Display the plot

def visualize_best():
    if len(generations[0][0]) != 2:
        values = []
        values = [objective_function1(binary_to_int(generations[i][0])) for i in range(len(generations))]
        plt.figure()  # Create a new figure for plotting
        plt.plot(values, marker = '.', markersize = 20)  # Plot the fitness values over generations
        plt.title("GA Generations Visualization - Objective Function 1")  # Set the title for the plot
        plt.xlabel("Best Individual of each Generation")  # Set the x-axis label
        plt.ylabel("Fitness Value")  # Set the y-axis label
        plt.show()  # Display the plot
    else:
        values = []
        values = [objective_function2(generations[i][0][0], generations[i][0][1]) for i in range(len(generations))]
        plt.figure()  # Create a new figure for plotting
        plt.plot(values, marker = '.', markersize = 20)  # Plot the fitness values over generations
        plt.title("GA Generations Visualization - Objective Function 2")  # Set the title for the plot
        plt.xlabel("Best Individual of each Generation")  # Set the x-axis label
        plt.ylabel("Fitness Value")  # Set the y-axis label
        plt.show()  # Display the plot

# Function to pick eligilbe random parents to be crossoveres
def pick_random_indices(mutated_indices, max_index):
    index_1, index_2 = -1, -1
    for i in range (2):
        rand = random.randint(0, max_index)
        while(mutated_indices.count(rand) > 0):
            if(rand >= max_index):
                rand = 0
            else:
                rand += 1
        if(i == 0):
            index_1 = rand
        else:
            index_2 = rand
    return index_1, index_2

# Function to generate a new generation from an old generation
def produce_generation(old_generation):
    new_generation = []  # Initialize an empty list for the new generation
    mutated_indices = [] # Initialize a list that store the indices of the parents that went through a crossover

    for i in range(0, (int)(len(old_generation) * crossover_rate) // 2): # Looping on the only population that will get crossovered
        index_1, index_2 = pick_random_indices(mutated_indices, (int)(len(old_generation) * crossover_rate - 1)) # Method to get two parents that have not been crossovered
        mutated_indices += [index_1] + [index_2] # Adding the chosen indices to the list of crossovered indices
        
        if len(old_generation[i]) != 2:  # Check the representation type (binary or real)
            child1, child2 = crossover(old_generation[index_1], old_generation[index_2], "one_point")  # Apply one-point crossover to these parents
            new_generation.append(child1) # Adding the children to the new generation
            new_generation.append(child2) # Adding the children to the new generation
        else:
            child1, child2 = crossover(old_generation[index_1][0], old_generation[index_2][1], "uniform")  # Apply uniform crossover
            child3, child4 = crossover(old_generation[index_1][1], old_generation[index_2][0], "uniform")  # Apply uniform crossover
            new_generation.append([child1, child3]) # Adding the children to the new generation
            new_generation.append([child4, child2]) # Adding the children to the new generation

    for i in range((int)(len(old_generation) * crossover_rate), len(old_generation)):
        new_generation.append(old_generation[i]) # Adding the rest of the parents that have been not crossovered due to crossover_rate

    mutated_generation = perform_mutation(new_generation)  # Apply mutation to the new generation

    return mutated_generation  # Return the new generation after crossover and mutation

# Function to perform mutation on a generation
def perform_mutation(old_generation):
    if len(old_generation[0]) != 2:  # Check the representation type (binary or real)
        for i in range(len(old_generation)):
            old_generation[i] = mutation(old_generation[i])  # Apply mutation to each individual
    else:
        for i in range(len(old_generation)):
            old_generation[i][0] = mutation_objective2(old_generation[i][0])  # Apply mutation to the first value of each individual
            old_generation[i][0] = mutation_objective2(old_generation[i][1])  # Apply mutation to the second value of each individual
    return old_generation  # Return the generation after mutation

def genetic_algorithm(first_generation):
    global generations  # Access the global variable 'generations'
    global counter  # Access the global variable 'counter'

    f_flag = False # It's a flag initiated to know if generations'fitness became to get worse
    g_flag = False # It's a flag initiated to know if generations'fitness have reached a global minimum or maximum

    if len(first_generation[0]) != 2:  # Check the representation type (binary or real) (Which objective function)
        first_generation_int = list_to_int(first_generation)  # Convert the binary values to integer values

    if len(first_generation[0]) != 2:  # Check the representation type (binary or real) (Which objective function)
        fitness_values = [objective_function1(individual) for individual in first_generation_int]
        visualize_generations1(fitness_values)  # Visualize fitness values for Objective Function 1
    else:
        fitness_values = [objective_function2(individual[0], individual[1]) for individual in first_generation]
        visualize_generations2(fitness_values)  # Visualize fitness values for Objective Function 2

    if len(generations[-1][0]) != 2: # Check the representation type (binary or real) (Which objective function)
        print(f"Generation {len(generations)} highest fitness value: ", objective_function1(binary_to_int(generations[-1][0])))
    else:
        print(f"Generation {len(generations)} highest fitness value: ",  objective_function2(generations[-1][0][0], generations[-1][0][1]))

    
    if len(first_generation[0]) != 2: # Check if it's the first or second objective function
        # This section checks the g_flag
        if(objective_function1(binary_to_int(generations[-1][0])) == global_maximum):
            g_flag = True
    else:
        if(objective_function2(generations[-1][0][0], generations[-1][0][1]) <= global_minimum):
            g_flag = True

    if(len(generations) >= generations_threshold):
        # This section checks the f_flag
        if len(generations[-1][0]) != 2:
            if objective_function1(binary_to_int(generations[-1][0])) < objective_function1(binary_to_int(generations[-2][0])):
                f_flag = True
                generations = generations[:-1]
        else:
            if objective_function2(generations[-1][0][0], generations[-1][0][1]) > objective_function2((generations[-2][0][0]), (generations[-2][0][1])):
                f_flag = True
                generations = generations[:-1]


    # This section checks if we finished producing generations
    if(len(generations) >= number_of_generations or f_flag or g_flag):  # Check if we have generated wanted number of generations generations
        print(f"{len(generations)+1} Generations are created")
        # print(generations)
        if len(generations[-1][0]) != 2: # Checks which objective function
            print("The best chromosome is: ", generations[-1][0])
            print("With fitness value: ", objective_function1(binary_to_int(generations[-1][0])))
        else:
            print("The best chromosome is: ", [(generations[-1][0][0]), (generations[-1][0][1])])
            print("With fitness value: ", objective_function2((generations[-1][0][0]), (generations[-1][0][1])))

        visualize_best()
        return  # End the recursive function if 5 generations are generated

    print("\n")

    new_generation = produce_generation(first_generation)  # Generate a new generation

    if len(new_generation[0]) != 2:  # Check the representation type (binary or real) (Which objective function)
        new_generation = list_to_gray(new_generation) # Put the generation in gray code
        new_generation = sort_binary_descending(new_generation)  # Sort the binary generation by fitness
    else:
        new_generation = sort_ascending(new_generation)  # Sort the real generation by fitness

    generations += [new_generation]  # Add the new generation to the list of generations
    counter += 1  # Increment the counter for the current generation
        

    genetic_algorithm(new_generation)  # Recursively call the function with the new generation

if __name__ == "__main__":
    func_1_init = initialize_population(population, 'real')  # Initialize the binary population. For objective function 1 type 'binary' and for 2 type 'real'

    if len(func_1_init[0]) != 2:
        func_1_init = list_to_gray(func_1_init)
        func_1_init = sort_binary_descending(func_1_init)  # Sort the binary population by fitness
        generations.append(func_1_init)
    else:
        func_1_init = sort_ascending(func_1_init)  # Sort the real population by fitness
        generations.append(func_1_init)

    genetic_algorithm(func_1_init)  # Start the genetic algorithm with the initial generation